# This is the first Final Assignment of the Applied Data Science 

**We will explore and cluster the neighborhoods in Toronto.**

1. First, we will **scrape** the data with neighborhoods in Toronto, including their Postal Code, Borough, and Neighborhoods, from the Wikipedia [Page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M).
2. We will use the data to create a clean **Pandas DataFrame** corresponding Postal Codes with Neighborhoods.
3. Then we use **geocoder** to extract coordinates for each Neighborhoods for the **Foursquare API** query.
4. Cluster the Neighborhoods  with **scikit-learn**, according to the venues located within a reasonable radius.
5. Present the map with clustered neighborhoods using **Folium**.

In [1]:
import pandas as pd
import numpy as np

import json # handle json files
import requests # send queries with urls
from urllib.request import urlopen
from lxml import html

from sklearn.cluster import KMeans # cluster the data

import folium # build maps

from bs4 import BeautifulSoup # scrape data

import matplotlib.cm as cm # color the final map
import matplotlib.colors as colors

## Part 1: Scrape the Data
Using the **requests** and **BeautifulSoup** libraries.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki = requests.get(url) # send GET request to the wiki page

soup = BeautifulSoup(wiki.text, 'html.parser') # convert the text to an html object
soup = soup.find_all('table') # find all tables in the page

table = soup[0] # take the first one, as it is the one with the data
print(type(table))

<class 'bs4.element.Tag'>


## Part 2: Create a Clean Pandas DataFrame
Using the **pandas** and **BeautifulSoup** libraries.

In [3]:
# Iterate over the wiki table and populate corresponding lists
postals = [] # list for Postal Codes
boroughs = [] # list for Boroughs
neighborhoods = [] # list for Neighborhoods
rows = table.find_all('tr') # find elements with the <tr> tag

for row in rows[1:]: # iterate over all those elements (except for the first one as it is empty)
    cells = row.find_all('td') # fine all elements with the <td> tag
    
    # check if Borough is not assigned or Neighborhood is not assigned (see the wiki table)
    if cells[1].text.strip() == 'Not assigned' or cells[2].text.strip() == 'Not assigned':
        continue
        
    postals.append(cells[0].text.strip()) # Add all information to corresponding lists
    boroughs.append(cells[1].text.strip())
    
    neighborhoods_list = cells[2].text.strip().split(',')    
    neighborhoods.append(neighborhoods_list[0].strip())
    
    if len(neighborhoods_list) > 1:
        i = len(neighborhoods_list) - 1
        while i != 1:
            postals.append(cells[0].text.strip())
            boroughs.append(cells[1].text.strip())
            neighborhoods.append(neighborhoods_list[i].strip())
            i -= 1
            
neighborhoods

['Parkwoods',
 'Victoria Village',
 'Regent Park',
 'Lawrence Manor',
 "Queen's Park",
 'Islington Avenue',
 'Malvern',
 'Don Mills',
 'Parkview Hill',
 'Garden District',
 'Glencairn',
 'West Deane Park',
 'Cloverdale',
 'Islington',
 'Martin Grove',
 'Rouge Hill',
 'Highland Creek',
 'Don Mills',
 'Woodbine Heights',
 'St. James Town',
 'Humewood-Cedarvale',
 'Eringate',
 'Markland Wood',
 'Old Burnhamthorpe',
 'Guildwood',
 'West Hill',
 'The Beaches',
 'Berczy Park',
 'Caledonia-Fairbanks',
 'Woburn',
 'Leaside',
 'Central Bay Street',
 'Christie',
 'Cedarbrae',
 'Hillcrest Village',
 'Bathurst Manor',
 'Downsview North',
 'Thorncliffe Park',
 'Richmond',
 'King',
 'Dufferin',
 'Scarborough Village',
 'Fairview',
 'Oriole',
 'Northwood Park',
 'East Toronto',
 'Harbourfront East',
 'Toronto Islands',
 'Little Portugal',
 'Kennedy Park',
 'East Birchmount Park',
 'Bayview Village',
 'Downsview',
 'The Danforth West',
 'Toronto Dominion Centre',
 'Brockton',
 'Exhibition Place',
 'Go

In [4]:
neighborhoods = pd.DataFrame({'Postal Code': postals, # Initialize a DataFrame
                             'Boroughs': boroughs,
                             'Neighborhoods': neighborhoods}) 
neighborhoods.head()

,Postal Code,Boroughs,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Manor
4,M7A,Downtown Toronto,Queen's Park


In [5]:
neighborhoods.shape

(153, 3)

## Part 3: Extract Coordinates for Each Neighborhood
Using the **geocoder** library. The **Goole Geocoding API** did not prove to work, so we will use the data provided by the course.

In [6]:
import geocoder # import geocoder

# WARNING: Does not return coordinates for a very long time
def get_lat_lng(postal_code):
    lat_lng_coords = None  # initialize your variable to None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(str(postal_code)))
      lat_lng_coords = g.latlng

    return lat_lng_coords

In [7]:
lat_lng_coords = pd.read_csv('Geospatial_Coordinates.csv') # read the given csv file
lat_lng_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
# merge two dataframes on their common postal codes
neighborhoods = pd.merge(neighborhoods, lat_lng_coords, on='Postal Code')
neighborhoods.head()

,Postal Code,Boroughs,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M6A,North York,Lawrence Manor,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


## Part 4: Cluster the Neighborhoods According to the Venues
Using the **scikit-learn** library and **Foursquare API**.
### Exploring the data set

In [9]:
print("There are {} unique boroughs".format(len(neighborhoods['Boroughs'].unique())))
print("and {} unique Postal Codes.".format(len(neighborhoods['Postal Code'].unique())))

There are 10 unique boroughs
and 103 unique Postal Codes.


In [10]:
# find the latitude and longitude of Toronto, ON
from geopy.geocoders import Nominatim
address = "Toronto, Canada"

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Toronto are {}, {}.".format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


In [11]:
# let's see all of the neighborhoods on the map
map_toronto = folium.Map(location=[latitude, longitude])

label = 'Center of Toronto, Canada'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude, longitude],
    radius=7,
    popup=label,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.7,
    ).add_to(map_toronto)

# add markers to the map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                          neighborhoods['Longitude'],
                                          neighborhoods['Boroughs'],
                                          neighborhoods['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False,
        ).add_to(map_toronto)

map_toronto

### We can see that the neighborhoods are scattered all across the city, but there is a **high density of neighborhoods** in the Downtown Toronto. 
We can see that **First Canadian Place**, **Toronto Dominion Centre**, and **Commerce Court** are practically within blocks of each other. There are many commercial venues in those neighborhoods, which probably means that they might be clustered together.

As it relates to other neighborhoods, the neighborhoods that are in the **Suburban Areas** of Toronto like **Humewood-Cedarvale, York** do not have many venues around them. Therefore, they will probably me clustered together.

### Now let's see what kind of data we will extract from the Foursquare API.
The **credentials** will be deactivated after this notebook is released.

In [12]:
CLIENT_ID = '5GNCFSAC2EKFQ1N0DIOUXNFK502JWGIUIRSXSYYZU5XFEX2Y' # your Foursquare ID
CLIENT_SECRET = '3UQQWIB0SP3GMJROGJERI1HIGZOCI04WX4HIQRJ3UNXZTVTC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5GNCFSAC2EKFQ1N0DIOUXNFK502JWGIUIRSXSYYZU5XFEX2Y
CLIENT_SECRET: 3UQQWIB0SP3GMJROGJERI1HIGZOCI04WX4HIQRJ3UNXZTVTC


### Let's first explore one of the neighborhood venues.
We will explore the **Parkwoods Neighborhood**.

In [13]:
neighborhood_name = neighborhoods.loc[0, 'Neighborhoods']
neighborhood_name

'Parkwoods'

In [14]:
neighborhood_lat = neighborhoods.loc[0, 'Latitude']
neighborhood_long = neighborhoods.loc[0, 'Longitude']


print('Latitude and longitude values of {}, Toronto are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_long))

Latitude and longitude values of Parkwoods, Toronto are 43.7532586, -79.3296565.


In [15]:
# building an explore Foursquare API query 
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_lat, neighborhood_long, 500 , LIMIT)

In [16]:
results = requests.get(url)
results = results.json()

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
venues = pd.json_normalize(venues)

# only take related columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues = venues.loc[:,filtered_columns]

# extract names of the categories
venues['venue.categories'] = venues.apply(get_category_type, axis=1)

# take only the last word in the column names
venues.columns = [col.split('.')[-1] for col in venues.columns]

venues

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,GreenWin pool,Pool,43.756232,-79.333842


In [19]:
print('{} venues were featured'.format(venues.shape[0]))

3 venues were featured


### Explore the rest of the neighborhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhoods'],
                                latitudes=neighborhoods['Latitude'],
                                longitudes=neighborhoods['Longitude'],)

Parkwoods
Victoria Village
Regent Park
Lawrence Manor
Queen's Park
Islington Avenue
Malvern
Don Mills
Parkview Hill
Garden District
Glencairn
West Deane Park
Cloverdale
Islington
Martin Grove
Rouge Hill
Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Thorncliffe Park
Richmond
King
Dufferin
Scarborough Village
Fairview
Oriole
Northwood Park
East Toronto
Harbourfront East
Toronto Islands
Little Portugal
Kennedy Park
East Birchmount Park
Bayview Village
Downsview
The Danforth West
Toronto Dominion Centre
Brockton
Exhibition Place
Golden Mile
Oakridge
York Mills
Downsview
India Bazaar
Commerce Court
North Park
Upwood Park
Humber Summit
Cliffside
Scarborough Village West
Willowdale
Downsview
Studio District
Bedford Park
Del Ray
Keelsdale and Silverthorn
Humber

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(2845, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,GreenWin pool,43.756232,-79.333842,Pool
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [23]:
# how many venues were returned for each neighborhood?
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Albion Gardens,9,9,9,9,9,9
Alderwood,8,8,8,8,8,8
Bathurst Manor,21,21,21,21,21,21
Bathurst Quay,17,17,17,17,17,17
...,...,...,...,...,...,...
Willowdale,40,40,40,40,40,40
Woburn,3,3,3,3,3,3
Woodbine Heights,8,8,8,8,8,8


In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


### Analyze each neighborhood venue categories

In [25]:
pd.set_option('display.max_columns', None)

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')

# adding neighborhood column to the front
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
index = toronto_onehot.columns.get_loc("Neighborhood")
fixed_columns = [toronto_onehot.columns[index]] + list(toronto_onehot.columns[:index]) + list(toronto_onehot.columns[index+1:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [26]:
toronto_onehot.shape

(2845, 270)

In [27]:
# Group by Neighborhoods and find the mean between all columns for each row
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0

In [28]:
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0

### The five top venues in each neighborhood.

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1                     Lounge   0.2
2               Skating Rink   0.2
3  Latin American Restaurant   0.2
4             Clothing Store   0.2


----Albion Gardens----
                  venue  freq
0         Grocery Store  0.22
1           Pizza Place  0.11
2        Discount Store  0.11
3        Sandwich Place  0.11
4  Fast Food Restaurant  0.11


----Alderwood----
            venue  freq
0     Pizza Place  0.25
1  Sandwich Place  0.12
2     Coffee Shop  0.12
3             Pub  0.12
4    Dance Studio  0.12


----Bathurst Manor----
                venue  freq
0                Bank  0.10
1         Coffee Shop  0.10
2         Supermarket  0.05
3  Chinese Restaurant  0.05
4          Restaurant  0.05


----Bathurst Quay----
                 venue  freq
0      Airport Service  0.18
1       Airport Lounge  0.12
2     Airport Terminal  0.12
3                Plane  0.06
4  Rental Car Location  0.06


----Bayvie

                 venue  freq
0  Arts & Crafts Store  0.08
1                 Park  0.08
2   Mexican Restaurant  0.08
3                 Café  0.08
4      Thai Restaurant  0.08


----Highland Creek----
                        venue  freq
0  Construction & Landscaping   0.5
1                         Bar   0.5
2           Accessories Store   0.0
3   Middle Eastern Restaurant   0.0
4         Monument / Landmark   0.0


----Hillcrest Village----
                      venue  freq
0               Golf Course   0.2
1      Fast Food Restaurant   0.2
2                   Dog Run   0.2
3  Mediterranean Restaurant   0.2
4                      Pool   0.2


----Humber Bay----
                        venue  freq
0  Construction & Landscaping   0.5
1              Baseball Field   0.5
2           Accessories Store   0.0
3   Middle Eastern Restaurant   0.0
4         Monument / Landmark   0.0


----Humber Bay Shores----
          venue  freq
0          Café  0.17
1   Pizza Place  0.08
2    Restaurant  0.08


                venue  freq
0                Park  0.33
1                Pool  0.33
2   Food & Drink Shop  0.33
3   Accessories Store  0.00
4  Mexican Restaurant  0.00


----Queen's Park----
              venue  freq
0       Coffee Shop  0.22
1  Sushi Restaurant  0.06
2               Bar  0.03
3          Beer Bar  0.03
4     Smoothie Shop  0.03


----Railway Lands----
                 venue  freq
0      Airport Service  0.18
1       Airport Lounge  0.12
2     Airport Terminal  0.12
3                Plane  0.06
4  Rental Car Location  0.06


----Regent Park----
         venue  freq
0  Coffee Shop  0.15
1         Café  0.06
2       Bakery  0.06
3          Pub  0.06
4         Park  0.06


----Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2            Hotel  0.04
3       Restaurant  0.04
4  Thai Restaurant  0.03


----Richview Gardens----
                       venue  freq
0                Pizza Place  0.25
1             Sandwich Place  0.25
2     

### The top ten venues in each neighborhood.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Clothing Store,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,Albion Gardens,Grocery Store,Pharmacy,Pizza Place,Fast Food Restaurant,Discount Store,Beer Store,Fried Chicken Joint,Sandwich Place,Yoga Studio,Dog Run
2,Alderwood,Pizza Place,Dance Studio,Gym,Pharmacy,Coffee Shop,Sandwich Place,Pub,Distribution Center,Dessert Shop,Dim Sum Restaurant
3,Bathurst Manor,Coffee Shop,Bank,Chinese Restaurant,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket,Deli / Bodega
4,Bathurst Quay,Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Rental Car Location,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court


### Time to cluster.

In [32]:
# set the number of clusters (groups)
kclusters = 5

# create new dataframe without neighborhood names
toronto_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering algorithms
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

kmeans.labels_[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [33]:
# add cluster labels to the DataFrame
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# rename the initial dataframe to have a neighborhood column for merge
neighborhoods.rename(columns={'Neighborhoods': 'Neighborhood'}, inplace=True)

# merge two DataFrames on common Neighborhood column
toronto_merged = pd.merge(neighborhoods, neighborhoods_venues_sorted, on='Neighborhood')
toronto_merged.head()

,Postal Code,Boroughs,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Food & Drink Shop,Park,Pool,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Beer Store,Brewery
3,M6A,North York,Lawrence Manor,43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Shoe Store,Miscellaneous Shop,Athletics & Sports,Carpet Store,Boutique,Vietnamese Restaurant
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Music Venue,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café


## Part 5: Present the Map with Clustered Labels

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Beer Store,Brewery
3,North York,0,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Shoe Store,Miscellaneous Shop,Athletics & Sports,Carpet Store,Boutique,Vietnamese Restaurant
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Music Venue,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café
5,Scarborough,0,Fast Food Restaurant,Print Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
134,East Toronto,0,Fast Food Restaurant,Light Rail Station,Butcher,Skate Park,Burrito Place,Garden,Garden Center,Spa,Farmers Market,Restaurant
142,Etobicoke,0,Grocery Store,Discount Store,Supplement Shop,Tanning Salon,Burrito Place,Burger Joint,Sandwich Place,Flower Shop,Bakery,Convenience Store
143,Etobicoke,0,Grocery Store,Discount Store,Supplement Shop,Tanning Salon,Burrito Place,Burger Joint,Sandwich Place,Flower Shop,Bakery,Convenience Store
144,Etobicoke,0,Grocery Store,Discount Store,Supplement Shop,Tanning Salon,Burrito Place,Burger Joint,Sandwich Place,Flower Shop,Bakery,Convenience Store


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
135,Etobicoke,1,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
136,Etobicoke,1,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
137,Etobicoke,1,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
138,Etobicoke,1,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
139,Etobicoke,1,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
140,Etobicoke,1,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
141,Etobicoke,1,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
131,Etobicoke,2,River,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Dessert Shop
132,Etobicoke,2,River,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Dessert Shop


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Food & Drink Shop,Park,Pool,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
24,York,3,Park,Women's Store,Pool,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
41,East York,3,Park,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
81,North York,3,Park,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
103,Scarborough,3,Park,Intersection,Playground,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
104,Scarborough,3,Park,Intersection,Playground,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
105,Scarborough,3,Park,Intersection,Playground,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
126,Downtown Toronto,3,Park,Trail,Playground,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Scarborough,4,Business Service,Playground,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
